Importing pandas library to do data scrapping from Wikipedia

In [1]:
import pandas as pd
import geocoder

In [2]:
#this code is from the previous notebook
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
table=pd.read_html(url)
df=pd.DataFrame(table[0])
df=df[df['Borough']!='Not assigned'].reset_index(drop=True)
idx=df[df['Neighbourhood']=='Not assigned'].index
if len(idx):
    for i in idx:
        df.loc[i,'Neighbourhood']=df.loc[i,'Borough']
df.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Since package can be very unreliable. And it's taking a lot of time to get latitude and logitude, I'll be using the csv file that has been provided.

In [3]:
latlang_data=pd.read_csv('H:/Data/Geospatial_Coordinates.csv')
latlang_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Now let's merge both dataframes on Postal Code.

In [4]:
data=df.merge(latlang_data,how='left',on='Postal Code')
print("New dataframe's shape is: ",data.shape)
data.head()

New dataframe's shape is:  (103, 5)


,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


Importing necessary libraries

In [6]:
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans
import folium

Since Toronto is Downtown Toronto we take all Downtown Toronto save it in dataToronto and resetting all the index of new data.

In [10]:
dataToronto=data[data['Borough']=='Downtown Toronto'].reset_index(drop=True)
#see 5 random row
dataToronto.sample(5)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
18,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
11,M5S,Downtown Toronto,"University of Toronto, Harbord",43.662696,-79.400049
7,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
9,M5K,Downtown Toronto,"Toronto Dominion Centre, Design Exchange",43.647177,-79.381576
8,M5J,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752


Let's get the geographical coordinates of Toronto.

In [19]:
geolocator = Nominatim(user_agent = "ca_explorer")
location = geolocator.geocode('Downtown Toronto')
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6563221, -79.3809161.


In [24]:
# create map of Manhattan using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, label in zip(dataToronto['Latitude'], dataToronto['Longitude'], dataToronto['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto